In [ ]:
### Uncomment the following lines to install the required packages###
# # For Microsoft Entra ID token authentication support 
# ! pip install azure-identity 
# # For Azure OpenAI Assistant 
# ! pip install "autogen-ext[openai]"
# # This module provides the main entry point for the autogen_agentchat package 
# ! pip install autogen-agentchat

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

In [ ]:
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

In [ ]:
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="<your deployment model name>",  # e.g., "gpt-4o"
    api_version="<model version>",  # e.g., "2023-05-15"
    model = "<your deployment model name>",  # e.g., "gpt-4o"
    azure_endpoint="<Azure OpenAI Service endpoint>",
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key="<Your API Key from AI foundry>", # For key-based authentication.
)

In [ ]:
# Create the story writer agent for kids to write a story based on user input.
Story_Writer = AssistantAgent(
    name="Story_Writer",
    description="You are story writer agent for kids to write a story based on user input. Keep the story simple and easy to understand.",
    model_client=az_model_client
)

In [ ]:
# Create the story reviewer agent
Story_Reviewer = AssistantAgent(
    name="Story_Reviewer",
    model_client=az_model_client,
    system_message="You are a helpful story reviewer assistant. You will review the story written by the Story_Writer agent and will provide critical feedbacks. Repond with 'Approved' when your feedbacks are addressed."
)

In [ ]:
#Define a termination condition if the story is approved
termination_condition = TextMentionTermination("Approved")
max_message_termination = MaxMessageTermination(5)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = termination_condition | max_message_termination

In [ ]:

team_chat = SelectorGroupChat(
    [Story_Writer, Story_Reviewer],
    model_client=az_model_client,
    termination_condition=termination
)



In [ ]:
async def main():
    
    await Console(
                    team_chat.run_stream(task="""

                    Write a story about a small girl who loves to play with her pet kitten. The story should be simple and easy to understand for kids. The story should be fun and engaging, with a happy ending.

        """)
    )  # Stream the messages to the console.

In [ ]:
await main()
    